In [1]:
import numpy as np
import random
from tqdm.auto import tqdm
import cv2
import os
import shutil

In [ ]:
# df = pd.read_csv("trainVal.csv")
# arrUnique = df['lp'].unique()
# ret = []
# for unique in tqdm(arrUnique):
#     uniqueIndexs = df[df['lp']==unique].index
#     index = random.randint(0,uniqueIndexs.size-1)
#     img_info = df.loc[uniqueIndexs[index]]
#     ret.append(np.array([img_info[1], img_info[2]]))
#     
# ret = np.array(ret)
# np.save("hlp-locations.npy", ret)

In [2]:
DIR = "Datasets/hlp"
DIR_TREINO = "Datasets/hlp/Treino"
DIR_TREINO2 = "Datasets/hlp/Treino2"

IMG_SIZE = 64

In [3]:
arr = np.load("hlp-locations.npy")


In [4]:
'''
    Separa placas com 7 digitos e que os 4 ultimos sejam numerais.
    arr -> [ [img_path, LP] ... ]
    arr_sem_letra -> [ [img_path, LP] ... ] 
'''
arr_sem_letra = []
for img in tqdm(arr):
    if len(img[1]) != 7:
        continue
    if img[1][-4:].isnumeric() is False:
        continue
    arr_sem_letra.append(img)
arr_sem_letra = np.array(arr_sem_letra)


In [5]:
np.save("hlp-locations-final.npy", arr_sem_letra)

In [6]:
arr = np.load("hlp-locations-final.npy")


In [7]:
'''
    Agrupa as placas que possuem outras placas com os mesmos 4 ultrimos digitos.
    arr -> [ [img_path, LP] ... ]
    unicos_entre_si -> { LP[-4:] : [ [img_path, LP] ...] ... }
    
'''
unicos_entre_si = {}
for img_info in tqdm(arr):
    if img_info[1][-4:] in unicos_entre_si:
        unicos_entre_si[img_info[1][-4:]].append(img_info)
    else:
        unicos_entre_si[img_info[1][-4:]] = [img_info]

In [8]:
'''
    Monta o numpy.array com o path e a license plate (LP) de cada placa em unicos_entre_si.
    Para os casos de chaves que possuem mais de um  valor, escolhe aleatoriamente apenas 1 de cada.
    i.e:
    {'4698' : [
                ['s01_l01/2_7.png','7C24698'],
                ['s01_l02/479_6.png','6B44698']
                ] } -seleciona randomicamente-> ['s01_l02/479_6.png','6B44698']
    {'2347' : [
                ['s01_l01/4_17.png', '5AR2347']
                ] } ----seleciona o unico-----> ['s01_l01/4_17.png', '5AR2347']
'''
arr_unico_entre_si = []
for img in tqdm(unicos_entre_si):
    index = random.randint(0, len(unicos_entre_si[img])-1)
    arr_unico_entre_si.append(unicos_entre_si[img][index])
arr_unico_entre_si = np.array(arr_unico_entre_si)

In [9]:
np.save("hlp-locations-final.npy", arr_unico_entre_si)


In [10]:
arr = np.load('hlp-locations-final.npy')

In [11]:

'''
    Copia as imagens das placas separadas em arr_unico_entre_si para uma pasta (Treino).
'''
for img in tqdm(arr):
    try:
        shutil.copy(os.path.join(DIR, img[0]), os.path.join(DIR_TREINO, f"{img[1]}.png"))
    except shutil.Error as err:
        print(err)

In [12]:
'''
    Recorta as imagens separadas na pasta Treino, separando e salvando somente os 4 ultimos digitos.
    Salva as imagens cortadas na pasta Treino2
'''
for img_name in tqdm(os.listdir(DIR_TREINO), desc="Imagens"):
    img = cv2.imread(os.path.join(DIR_TREINO, img_name))
    num = []
    for i in range(img.shape[0]):
        tmp1 = []
        for j in range(int(img.shape[1]/2), img.shape[1]):
            tmp1.append(img[i][j])
        num.append(tmp1)
        
    num = np.array(num)
    cv2.imwrite(os.path.join(DIR_TREINO2, f"{img_name[:-4][-4:]}.png"), num)
    



In [4]:
'''
    Gera um numpy.array com a imagem e a label da imagem para facilitar o backup (salvar apenas um numpy.array
    com os dados, ao inves de todas as 5000+ imagens). 
    Imagens já com resize e cv2.IMREAD_GRAYSCALE
    
    img_array -> [ [imread(img), label], ...]
'''
img_array = []
for img_name in tqdm(os.listdir(DIR_TREINO2)):
    img = cv2.resize(cv2.imread(os.path.join(DIR_TREINO2, img_name), cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
    label = img_name[:-4]
    img_array.append([img, label])

img_array = np.array(img_array)
    

In [6]:
np.save(f"hlp-dataset-{img_array.shape[0]}-GrayScale-({IMG_SIZE},{IMG_SIZE}).npy", img_array)

In [ ]:
def visualizarImagem(img, label):
    cv2.imshow(label, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
